In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from mtcnn import MTCNN
from deepface import DeepFace
import torch
import matplotlib
matplotlib.use('TkAgg')  # Set backend before importing pyplot
import matplotlib.pyplot as plt
import time

# Funcții de preprocesare a imaginii
def apply_clahe(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    hsv[:, :, 2] = clahe.apply(hsv[:, :, 2])
    img_clahe = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
    return img_clahe

def apply_gamma_correction(img, gamma=1.5):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(256)]).astype("uint8")
    img_gamma = cv2.LUT(img, table)
    return img_gamma

# Verifică dacă GPU-ul este disponibil
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

# Initializează detectorul MTCNN fără puncte cheie
detector = MTCNN()

# Inițializează captura video
cap = cv2.VideoCapture(0)  # 0 pentru camera implicită
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Definirea emoțiilor în engleză (conform FER Plus)
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Încărcarea modelului de detectare a emoțiilor
model_path = 'model_emotion.h5'  # Asigură-te că ai modelul în acest path
try:
    model = load_model(model_path)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

print("Starting video stream. Press 'q' to exit.")

# Initialize Matplotlib pentru plotare în timp real
plt.ion()
fig, ax = plt.subplots()
emotion_counts = {label: 0 for label in emotion_labels}
bars = ax.bar(emotion_labels, [0]*len(emotion_labels), color='skyblue')
ax.set_ylim(0, 10)  # Limită inițială a axei Y
ax.set_ylabel('Count')
ax.set_title('Emotion Counts Over Time')
plt.show()

frame_counter = 0
start_time = time.time()

def get_largest_face(faces):
    if not faces:
        return None
    largest_face = max(faces, key=lambda face: face['box'][2] * face['box'][3])
    return largest_face

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to read frame from camera.")
        break

    # Aplică preprocesarea: CLAHE și corecție gamma
    processed_frame = apply_clahe(frame)
    processed_frame = apply_gamma_correction(processed_frame, gamma=1.5)

    # Detectare fețe cu MTCNN
    faces = detector.detect_faces(processed_frame)
    largest_face = get_largest_face(faces)

    # Creare mască pentru a blura fundalul
    mask = np.zeros_like(processed_frame)
    if largest_face:
        x, y, w, h = largest_face['box']
        x, y = max(0, x), max(0, y)
        w, h = min(w, processed_frame.shape[1] - x), min(h, processed_frame.shape[0] - y)
        cv2.rectangle(mask, (x, y), (x + w, y + h), (255, 255, 255), -1)

    # Aplicare blur pe fundal
    blurred_frame = cv2.GaussianBlur(processed_frame, (21, 21), 0)
    # Combina blurred_frame și processed_frame folosind masca
    combined_frame = np.where(mask == np.array([255, 255, 255]), processed_frame, blurred_frame)

    current_time = time.time()
    if current_time - start_time >= 1:
        if largest_face:
            try:
                face_roi_original = combined_frame[y:y + h, x:x + w].copy()
                face_gray = cv2.cvtColor(face_roi_original, cv2.COLOR_BGR2GRAY)
                face_eq = cv2.equalizeHist(face_gray)
                face_eq = cv2.GaussianBlur(face_eq, (3, 3), 0)
                face_roi = cv2.resize(face_eq, (48, 48))
                face_roi = face_roi.astype('float32') / 255.0
                face_roi = np.expand_dims(face_roi, axis=-1)
                face_roi = np.expand_dims(face_roi, axis=0)

                # Predicție emoție folosind modelul antrenat
                predictions = model.predict(face_roi)[0]
                if not isinstance(predictions, np.ndarray):
                    predictions = np.array(predictions)
                predictions = predictions / np.sum(predictions)

                # Predicție emoție folosind DeepFace
                face_roi_rgb = cv2.cvtColor(face_roi_original, cv2.COLOR_BGR2RGB)
                deepface_result = DeepFace.analyze(face_roi_rgb, actions=['emotion'], enforce_detection=False)

                # Extrage probabilitățile emoțiilor din DeepFace
                if isinstance(deepface_result, list):
                    if len(deepface_result) > 0:
                        deepface_emotions = deepface_result[0].get('emotion', {})
                    else:
                        deepface_emotions = {}
                elif isinstance(deepface_result, dict):
                    deepface_emotions = deepface_result.get('emotion', {})
                else:
                    deepface_emotions = {}

                # Maparea etichetelor DeepFace la cele din modelul tău personalizat
                deepface_emotions = {k.capitalize(): v for k, v in deepface_emotions.items()}
                deepface_mapped = {
                    'Angry': deepface_emotions.get('Angry', 0.0),
                    'Disgust': deepface_emotions.get('Disgust', 0.0),
                    'Fear': deepface_emotions.get('Fear', 0.0),
                    'Happy': deepface_emotions.get('Happy', 0.0),
                    'Sad': deepface_emotions.get('Sad', 0.0),
                    'Surprise': deepface_emotions.get('Surprise', 0.0),
                    'Neutral': deepface_emotions.get('Neutral', 0.0)
                }

                deepface_predictions = np.array([deepface_mapped[label] for label in emotion_labels])
                if np.sum(deepface_predictions) > 0:
                    deepface_predictions = deepface_predictions / np.sum(deepface_predictions)
                else:
                    deepface_predictions = np.zeros(len(emotion_labels))

                # Calcularea mediei probabilităților
                combined_predictions = (predictions + deepface_predictions) / 2
                final_max_index = np.argmax(combined_predictions)
                final_emotion = emotion_labels[final_max_index]
                emotion_counts[final_emotion] += 1

                # Desenare dreptunghi pe față fără acuratețe
                color = (255, 0, 0)  # Albastru
                cv2.rectangle(combined_frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(combined_frame, final_emotion, (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

                # Actualizare grafic
                for bar, label in zip(bars, emotion_labels):
                    bar.set_height(emotion_counts[label])
                max_count = max(emotion_counts.values()) if emotion_counts.values() else 10
                ax.set_ylim(0, max(max_count + 5, 10))
                fig.canvas.draw()
                fig.canvas.flush_events()
                plt.pause(0.001)

            except Exception as e:
                print("Error processing face:", e)
                pass

        start_time = current_time

    cv2.imshow('Original Frame', frame)
    cv2.imshow('Processed Frame - Blurred Background and Emotion', combined_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



Using device: cpu


Model loaded successfully.
Starting video stream. Press 'q' to exit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━

In [ ]:
##SECOND APPRPOACH
import cv2
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib
matplotlib.use('TkAgg')  # Set the Matplotlib backend before importing pyplot
import matplotlib.pyplot as plt
import time

# -------------------------------------------------------------------------
# 1. CUSTOM IMPORTS FOR YOUR MODELS
# -------------------------------------------------------------------------
# Assume you have:
#   - face_model.py (a PyTorch face detection model pretrained on DarkFace)
#   - emotion_model.py (a PyTorch emotion recognition model)
#
# Import them here. For example:
#
# from face_model import MyDarkFaceDetector  # your custom class
# from emotion_model import MyEmotionModel   # your custom class
#
# In this demo, we'll just define placeholders below for illustration.

# Placeholder classes (replace with your actual implementations!)
class MyDarkFaceDetector(torch.nn.Module):
    """
    Example: A face detection model pretrained on DarkFace dataset.
    Must return 'boxes' and 'scores' in forward() for each image.
    """
    def __init__(self):
        super().__init__()
        # define your architecture

    def forward(self, images):
        # images: [batch_size, 3, H, W] (torch.Tensor)
        # Return a list[dict], each dict with 'boxes' & 'scores'
        # boxes: [N, 4], scores: [N] ...
        return [{
            'boxes': torch.tensor([[100, 100, 200, 200]]),
            'scores': torch.tensor([0.99])
        }]


class MyEmotionModel(torch.nn.Module):
    """
    Example: An emotion recognition model
    input -> ... -> output logits for each emotion class
    """
    def __init__(self, num_emotions=7):
        super().__init__()
        # define your architecture

    def forward(self, x):
        # x: [batch_size, 1 (or 3), H, W], e.g., 48x48 if grayscale
        # Return logits for each emotion
        return torch.randn(x.size(0), 7)  # random placeholder

# -------------------------------------------------------------------------
# 2. IMAGE PREPROCESSING FUNCTIONS
# -------------------------------------------------------------------------
def apply_clahe(img):
    """
    Applies CLAHE (Contrast Limited Adaptive Histogram Equalization)
    to the Value channel in HSV space, which can enhance local contrast.
    """
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    hsv[:, :, 2] = clahe.apply(hsv[:, :, 2])  # apply on V channel
    img_clahe = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return img_clahe

def apply_gamma_correction(img, gamma=1.5):
    """
    Applies gamma correction to adjust brightness in a non-linear way.
    gamma > 1.0 darkens the image, gamma < 1.0 lightens it.
    """
    invGamma = 1.0 / gamma
    table = np.array([
        ((i / 255.0) ** invGamma) * 255 for i in np.arange(256)
    ]).astype("uint8")
    return cv2.LUT(img, table)

# -------------------------------------------------------------------------
# 3. UTILITY FUNCTIONS
# -------------------------------------------------------------------------
def largest_box(boxes, scores, score_threshold=0.5):
    """
    Given 'boxes' (N,4) and 'scores' (N), returns the bounding box
    with the largest area among those that exceed 'score_threshold'.
    Each box is [x_min, y_min, x_max, y_max].
    Returns None if no box meets the threshold.
    """
    valid_indices = (scores >= score_threshold).nonzero(as_tuple=True)[0]
    if len(valid_indices) == 0:
        return None
    
    # compute areas
    areas = []
    for i in valid_indices:
        x1, y1, x2, y2 = boxes[i]
        areas.append((x2 - x1) * (y2 - y1))
    
    max_idx = valid_indices[torch.argmax(torch.tensor(areas))]
    return boxes[max_idx].tolist()  # convert to Python list

# -------------------------------------------------------------------------
# 4. LOAD YOUR PRETRAINED MODELS (FACE DETECTOR & EMOTION MODEL)
# -------------------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Replace these with your real model definitions & weights
face_detector = MyDarkFaceDetector().to(device)
emotion_model = MyEmotionModel(num_emotions=7).to(device)

# Example: load state dicts from files
# face_detector.load_state_dict(torch.load("face_detector_darkface.pth", map_location=device))
# emotion_model.load_state_dict(torch.load("emotion_model.pth", map_location=device))

face_detector.eval()
emotion_model.eval()

# -------------------------------------------------------------------------
# 5. SET UP WEBCAM CAPTURE
# -------------------------------------------------------------------------
cap = cv2.VideoCapture(0)  # 0 = default webcam
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Define your emotion labels (English)
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# -------------------------------------------------------------------------
# MATPLOTLIB FOR REAL-TIME BAR CHART
# -------------------------------------------------------------------------
plt.ion()
fig, ax = plt.subplots()
emotion_counts = {label: 0 for label in emotion_labels}
bars = ax.bar(emotion_labels, [0]*len(emotion_labels), color='skyblue')
ax.set_ylim(0, 10)
ax.set_ylabel('Count')
ax.set_title('Emotion Counts Over Time')
plt.show()

start_time = time.time()
update_interval = 1.0  # seconds

print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to read frame from camera.")
        break
    
    # ---------------------------------------------------------------------
    # 6. IMAGE PREPROCESSING (CLAHE, GAMMA)
    # ---------------------------------------------------------------------
    # Convert to BGR if needed; usually cap.read() is already BGR
    processed_frame = apply_clahe(frame)
    processed_frame = apply_gamma_correction(processed_frame, gamma=1.5)

    # ---------------------------------------------------------------------
    # 7. FACE DETECTION WITH CUSTOM MODEL (DARKFACE PRETRAINED)
    # ---------------------------------------------------------------------
    # Prepare the frame for your face detector:
    # Usually you do transforms, e.g. ToTensor(), etc.
    # We'll do a minimal approach here:
    input_tensor = torch.from_numpy(processed_frame.copy()).permute(2, 0, 1).float().unsqueeze(0)
    input_tensor = input_tensor.to(device)
    
    # Forward pass
    with torch.no_grad():
        detections = face_detector(input_tensor)  # expected to return list of dicts

    # We assume the first element of detections is a dict with 'boxes' & 'scores'
    boxes = detections[0]['boxes']  # shape: [N, 4]
    scores = detections[0]['scores']  # shape: [N]
    
    # Find the largest face above a certain threshold
    best_box = largest_box(boxes, scores, score_threshold=0.5)
    
    # Create a mask for background blur
    mask = np.zeros_like(processed_frame, dtype=np.uint8)
    
    if best_box is not None:
        x1, y1, x2, y2 = map(int, best_box)
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(x2, processed_frame.shape[1]-1), min(y2, processed_frame.shape[0]-1)
        
        cv2.rectangle(mask, (x1, y1), (x2, y2), (255,255,255), -1)
    
    # Blur background
    blurred_frame = cv2.GaussianBlur(processed_frame, (21,21), 0)
    combined_frame = np.where(mask == np.array([255,255,255]), processed_frame, blurred_frame)

    current_time = time.time()
    if current_time - start_time >= update_interval:
        if best_box is not None:
            try:
                face_roi = combined_frame[y1:y2, x1:x2].copy()
                # Convert to grayscale (if your emotion model expects grayscale)
                face_gray = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)

                # Possibly do additional preprocessing (e.g., resize to 48x48)
                face_resized = cv2.resize(face_gray, (48, 48))  # adapt to your model
                face_resized = face_resized.astype(np.float32) / 255.0
                # shape: [48,48]
                face_resized = np.expand_dims(face_resized, axis=0)   # [1,48,48]
                face_resized = np.expand_dims(face_resized, axis=0)   # [1,1,48,48]
                
                face_tensor = torch.from_numpy(face_resized).to(device)
                
                # Forward pass through your emotion model
                with torch.no_grad():
                    logits = emotion_model(face_tensor)
                    probs = F.softmax(logits, dim=1)  # shape: [1,7]
                probs_np = probs.cpu().numpy()[0]
                
                emotion_idx = np.argmax(probs_np)
                emotion_str = emotion_labels[emotion_idx]
                
                # Increment count for this emotion
                emotion_counts[emotion_str] += 1
                
                # Draw bounding box and label
                color = (255, 0, 0)  # blue
                cv2.rectangle(combined_frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(combined_frame, emotion_str, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
                
                # Update bar chart
                for bar, label in zip(bars, emotion_labels):
                    bar.set_height(emotion_counts[label])
                max_count = max(emotion_counts.values()) if emotion_counts else 10
                ax.set_ylim(0, max(max_count + 5, 10))
                fig.canvas.draw()
                fig.canvas.flush_events()
                plt.pause(0.001)
            
            except Exception as e:
                print("Error processing face:", e)
        
        start_time = current_time
    
    # Display frames
    cv2.imshow("Original", frame)
    cv2.imshow("Processed + Blur", combined_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
